# Testing if the model works and can play a game against me

In [1]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
import helper_functions as helper
from torch.utils.data import DataLoader, random_split # type: ignore
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore
from dataset import ChessPGNDataset
from model import ChessNet, ResBlock

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [ ]:
import chess

# Load the move map
MOVE_MAP = helper.create_move_map()
MOVE_MAP_LENGTH = len(MOVE_MAP)

# Create reverse move map (index -> move_uci)
REVERSE_MOVE_MAP = {v: k for k, v in MOVE_MAP.items()}

print(f"Move map size: {MOVE_MAP_LENGTH}")


In [ ]:
# Load the trained model
model = ChessNet(MOVE_MAP_LENGTH)
model.load_state_dict(torch.load("models/supervised_learning_chess_model_1.pth", map_location=device))
model.to(device)
model.eval()

print("Model loaded and ready for inference!")


## Example: Get a legal move from the model

The three key helper functions work together:
1. `get_legal_moves_mask()` - Creates a mask of legal moves for the current position
2. `apply_legal_moves_mask()` - Applies the mask to model predictions
3. `get_best_legal_move()` - Combines everything to select a legal move


In [ ]:
# Create a starting position
board = chess.Board()

# Convert board to tensor
board_tensor = helper.turn_board_to_tensor(board).to(device)

# Get model prediction
with torch.no_grad():
    policy, value = model(board_tensor)

# Get a legal move with temperature control
# temperature=1.0 is normal softmax, <1.0 is more greedy, >1.0 is more random
move, confidence = helper.get_best_legal_move(
    board=board,
    policy_logits=policy,
    move_map=MOVE_MAP,
    reverse_move_map=REVERSE_MOVE_MAP,
    device=device,
    temperature=1.0
)

print(f"Board position:\n{board}")
print(f"\nLegal moves available: {len(list(board.legal_moves))}")
print(f"Model chose: {move} (confidence: {confidence:.4f})")
print(f"Is this move legal? {move in board.legal_moves}")


## Example: Play a full game


In [ ]:
# Play a complete game (model vs random opponent)
board = chess.Board()
move_count = 0
max_moves = 200

print("Starting game: Model (White) vs Random (Black)\n")

while not board.is_game_over() and move_count < max_moves:
    # Model's turn (White)
    if board.turn == chess.WHITE:
        board_tensor = helper.turn_board_to_tensor(board).to(device)
        with torch.no_grad():
            policy, value = model(board_tensor)
        move, conf = helper.get_best_legal_move(
            board=board,
            policy_logits=policy,
            move_map=MOVE_MAP,
            reverse_move_map=REVERSE_MOVE_MAP,
            device=device,
            temperature=0.5  # Slightly greedy
        )
        print(f"Move {move_count + 1}. White: {move} (confidence: {conf:.4f})")
    else:
        # Random opponent's turn (Black)
        move = chess.Move.from_uci(str(chess.Board().legal_moves.__iter__().__next__()))
        # Actually pick a random legal move
        import random
        move = random.choice(list(board.legal_moves))
        print(f"Move {move_count + 1}... Black: {move}")
    
    board.push(move)
    move_count += 1

print(f"\nGame over! Result: {board.result()}")
print(f"Moves played: {move_count}")


## Understanding Temperature Parameter

The `temperature` parameter controls how deterministic vs random the move selection is:

- **temperature < 1.0** (e.g., 0.1): Very greedy - picks the model's highest-confidence move almost always
- **temperature = 1.0**: Normal softmax - balanced random sampling from the learned distribution
- **temperature > 1.0** (e.g., 2.0): Very random - flattens probabilities, explores diverse moves

Higher temperature can help avoid getting stuck in local optima during self-play training.
